# ETL Pipeline - Generated Code
        
**Generated:** 2025-09-04 19:06:28  
**Configuration:** Text: Text file (7656 chars)

## Overview
This notebook contains the auto-generated ETL pipeline code for migrating data from Oracle to Databricks Delta Lake.


In [ ]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnull

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_spark_session():
    """Create a SparkSession with Delta Lake extensions"""
    spark = SparkSession.builder \
        .appName("Sales ETL Pipeline") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
    return spark

def load_data(spark):
    """Load data from database tables"""
    try:
        # Load customers table
        customers_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{os.environ['SALES_DB_HOST']}:{os.environ['SALES_DB_PORT']}/{os.environ['SALES_DB_NAME']}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "CUSTOMERS") \
            .option("user", os.environ['SALES_DB_USERNAME']) \
            .option("password", os.environ['SALES_DB_PASSWORD']) \
            .load()
        
        # Filter customers by status
        customers_df = customers_df.filter(col("STATUS") == "ACTIVE")
        
        # Load products table
        products_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{os.environ['SALES_DB_HOST']}:{os.environ['SALES_DB_PORT']}/{os.environ['SALES_DB_NAME']}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "PRODUCTS") \
            .option("user", os.environ['SALES_DB_USERNAME']) \
            .option("password", os.environ['SALES_DB_PASSWORD']) \
            .load()
        
        # Load sales table
        sales_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{os.environ['SALES_DB_HOST']}:{os.environ['SALES_DB_PORT']}/{os.environ['SALES_DB_NAME']}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "SALES") \
            .option("user", os.environ['SALES_DB_USERNAME']) \
            .option("password", os.environ['SALES_DB_PASSWORD']) \
            .load()
        
        # Filter sales by quantity and total amount
        sales_df = sales_df.filter((col("QUANTITY") > 0) & (col("TOTAL_AMOUNT") > 0))
        
        return customers_df, products_df, sales_df
    
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

def transform_data(customers_df, products_df, sales_df):
    """Transform data by joining tables and aggregating sales"""
    try:
        # Join sales with customers and products using broadcast
        sales_df = sales_df.join(broadcast(customers_df), "CUSTOMER_ID", "inner")
        sales_df = sales_df.join(broadcast(products_df), "PRODUCT_ID", "inner")
        
        # Aggregate sales by customer and product
        sales_agg_df = sales_df.groupBy("CUSTOMER_ID", "CUSTOMER_NAME", "PRODUCT_ID", "PRODUCT_NAME", "SALE_DATE") \
            .agg(count("SALE_ID").alias("SALES_COUNT"), sum("TOTAL_AMOUNT").alias("TOTAL_SALES"))
        
        return sales_agg_df
    
    except Exception as e:
        logger.error(f"Error transforming data: {e}")
        raise

def load_data_to_delta(sales_agg_df):
    """Load transformed data to Delta Lake"""
    try:
        # Write data to Delta Lake with partitioning
        sales_agg_df.write.format("delta") \
            .partitionBy("SALE_DATE") \
            .mode("overwrite") \
            .saveAsTable("SALES_AGGREGATED")
        
        logger.info("Data loaded to Delta Lake successfully")
    
    except Exception as e:
        logger.error(f"Error loading data to Delta Lake: {e}")
        raise

def data_quality_checks(sales_agg_df):
    """Perform data quality checks"""
    try:
        # Check for null values
        null_counts = sales_agg_df.select([count(isnull(c)).alias(c) for c in sales_agg_df.columns]).collect()
        logger.info("Null counts:")
        for row in null_counts:
            for col_name, null_count in row.asDict().items():
                logger.info(f"{col_name}: {null_count}")
        
        # Check row count
        row_count = sales_agg_df.count()
        logger.info(f"Row count: {row_count}")
    
    except Exception as e:
        logger.error(f"Error performing data quality checks: {e}")
        raise

def main():
    spark = create_spark_session()
    logger.info("SparkSession created successfully")
    
    customers_df, products_df, sales_df = load_data(spark)
    logger.info("Data loaded successfully")
    
    sales_agg_df = transform_data(customers_df, products_df, sales_df)
    logger.info("Data transformed successfully")
    
    load_data_to_delta(sales_agg_df)
    logger.info("Data loaded to Delta Lake successfully")
    
    data_quality_checks(sales_agg_df)
    logger.info("Data quality checks completed successfully")

if __name__ == "__main__":
    main()

## Validation Report

**Summary:** 9/9 checks passed

| Check | Status | Details |
|-------|--------|---------|
| SparkSession | PASS PASS | SparkSession properly initialized |
| Delta Lake | PASS PASS | Delta Lake format detected |
| Environment Variables | PASS PASS | Uses environment variables |
| No Hardcoded Creds | PASS PASS | No hardcoded credentials found |
| Predicate Pushdown | PASS PASS | Database-level filtering detected |
| Broadcast Joins | PASS PASS | Broadcast joins implemented |
| Error Handling | PASS PASS | Exception handling present |
| Logging | PASS PASS | Logging implemented |
| Data Quality Checks | PASS PASS | Data quality checks present |


## Test Report

**Summary:** 5/6 tests passed

| Test | Status | Input | Expected | Output |
|------|--------|-------|----------|--------|
| Syntax Validation | PASS | Python code compilation | Valid Python syntax | Code compiles successfully |
| Business Rules Filter | PASS | 3 records with mixed status/values | 1 valid record | 1 records after filtering |
| Data Transformation | PASS | Sales with dates | Year/month extraction | 2 unique year-month combinations |
| Aggregation Logic | PASS | 4 records to aggregate | Customer 1, Product 10: qty=8, amt=80 | Aggregation produces 3 groups |
| Data Volume Handling | PASS | Simulated 1,000,000 records | Handles large volumes | Volume test passed |
| Performance Optimizations | FAIL | Code analysis | Performance features | Found: broadcast joins |
